# Importar las librerías necesarias

In [16]:
import pandas as pd
import openpyxl

# Descargar el archivo de inventario, o crear uno nuevo

In [17]:
def gen_file():
  """
  Función para la descarga o creación de un archivo de inventario.
  """
  
  #Solicitar si se quiere descargar un archivo o crear uno nuevo
  soli = input("¿Descargar archivo o crear nuevo archivo? (D/N) ")
  soli = soli.upper()

  df = None

  #Verificar el resultado de la pregunta
  if soli == "D":
    while True:
      #Ruta del archivo necesario en Google Drive
      file_path = input("Ingrese el nombre del archivo: ")

      #Se busca el archivo y se intenta acceder
      try:
        df = pd.read_excel(f"{file_path}.xlsx")
        print("Archivo encontrado.")
        break

      #En caso de no ser posible, se avisa al usuario
      except:
        print("No se pudo leer el archivo.")
        
        #Comprobar si se quiere volver a intentar
        verif = input("¿Desea volver a intentarlo? (Y/N): ")
        verif = verif.upper()

        #En caso de que no se desee reintentar
        if verif == "N":
          break

  #Generar la tabla de datos
  elif soli == "N":
    #Ruta para guardar el archivo
    file_path = input("Ingrese el nombre del documento que se desea crear: ")

    while True:

      #Solicitar la información del producto
      print("Introduzca los datos del primer producto que desea guardar.")
      code = input("Ingrese el código del producto: ")
      name = input("Ingrese el nombre del producto: ")
      buy = input("Ingrese la cantidad de productos comprados: ")
      sell = input("Ingrese la cantidad de productos vendidos: ")
      damage = input("Ingrese la cantidad de productos dañados: ")

      try:
        #Intentar convertir los datos de compra, venta y daño en números
        buy = int(buy)
        sell = int(sell)
        damage = int(damage)
        stock = buy - sell - damage

        #Mostrar los datos para verificación
        print("Verifique los datos")
        print(f"Código: {code}")
        print(f"Nombre: {name}")
        print(f"Compra:  {buy}")
        print(f"Venta:  {sell}")
        print(f"Dañado: {damage}")

        #Confirmar que los datos sean correctos
        accept = input("¿Son los datos correctos? (Y/N) ")
        print()
        accept = accept.upper()

        #Verificar la respuesta dada
        if accept == "Y":
          #En caso de que los datos sean correctos
          break

      except ValueError:
        #En caso de un error al intentar lo anterior
        print("Alguno de los datos no es válido.")

        #Verificar si se desea volver a intentar crear el archivo
        verif = input("¿Desea volver a intentarlo? (Y/N): ")
        verif = verif.upper

        #En caso de que no desee repetir, finalizar la función
        if verif == "N":
          break

      else:
        print("Ha ocurrido un error inesperado.")

    cols = ["Código", "Nombre", "Compra", "Venta", "Dañado", "Stock"]
    df = [[code, name, buy, sell, damage, stock]]
    df = pd.DataFrame(df, columns=cols)

    #Guardar el documento
    df.to_excel(f"{file_path}.xlsx", index=False, header=False)

  #En caso de que el dato ingresado no corresponda con las opciones
  else:
    print("Opción no reconocida.")

  return df, f"{file_path}.xlsx"

# Funciones para editar la tabla

## Guardar cambios

In [19]:
#Función para guardar los cambios en el archivo original (fuente)
def Save_Changes():
  global df, file_path
  return df.to_excel(file_path, index=False)

## Rehacer la tabla, específicamente las celdas de stock

In [20]:
#Función para mostrar los cambios en el stock, en el archivo local
def refresh():
  global df
  df['Stock'] = df['Compra'] - df['Venta'] - df['Dañado']
  return df["Stock"]

## Añadir nueva fila de producto

In [21]:
#Función para añadir filas
def new_product():
  global df

  while True:
    #Solicitar datos del producto
    code = input("Ingrese el código del producto: ")
    name = input("Ingrese el nombre del producto: ")
    buy = input("Ingrese la cantidad de productos comprados: ")
    sell = input("Ingrese la cantidad de productos vendidos: ")
    damage = input("Ingrese la cantidad de productos dañados: ")

    try:
      #Convertir los datos de compra, venta y daño en números
      buy = int(buy)
      sell = int(sell)
      damage = int(damage)
      stock = buy - sell - damage

      #Generar la fila
      cols = ["Código", "Nombre", "Compra", "Venta", "Dañado", "Stock"]
      nuevo_producto = pd.DataFrame([[code, name, buy, sell, damage, stock]], columns=cols)
      
      #Agregar la fila a la tabla
      df = pd.concat([df, nuevo_producto], ignore_index=True)
      
      break

    except:
      #En caso de que alguno de los datos no sea válido
      print("Alguno de los datos no es válido.")

      #Preguntar si quiere volver a ingresar los datos
      verif = input("¿Desea volver a ingresar los datos? (Y/N): ")
      verif = verif.upper()

      #En caso de que no desee volver a ingresar datos
      if verif == "N":
        break

  #Refrescar la tabla
  refresh()

  #Mostrar la tabla
  display(df)

## Eliminar un producto

In [22]:
#Función para añadir filas
def delete():
  global df
  
  while True:
    #Solicitar datos del producto
    code = input("Ingrese el código del producto: ")

    try:
      #Generar la tabla de nuevo, con los productos de código diferente
      df = df[df['Código'] != code]

      break

    except:
      #Mostrar el error al buscar el código
      print("No se ha encontrado el código del producto.")

      #Preguntar si quiere volver a intentar buscar el código
      verif = input("¿Desea volver a ingresar el código? (Y/N): ")
      verif = verif.upper()

      #Si no desea ingresar código, termina la función
      if verif == "N":
        break

  #Refrescar la tabla
  refresh()

  #Mostrar la tabla
  display(df)

## Ingresar valores de compra

In [23]:
#Función para editar los valores de producto comprado, en el archivo local
def Buy():
  global df

  while True:
    #Solicitud de la cantidad de producto comprado
    cant = input("Ingrese la cantidad comprada: ")
    try:
      cant = int(cant)

      break

    except:
      print("Cantidad no reconocida.")
      
      verif = input("¿Desea volver a intentarlo? (Y/N): ")
      verif = verif.upper()

      if verif == "N":
        break

    #Solicitud de código del producto comprado
    code = input("Ingrese el código del producto: ")
    code = code.upper()
    
    #Actualización de la cantidad del producto comprado
    try:
      df.loc[df['Código'] == code, 'Compra'] += cant

      break

    except:
      print("Código no encontrado.")

      verif = input("¿Desea volver a intentarlo? (Y/N):")
      verif = verif.upper

      if verif == "N":
        break

  #Refrescar la tabla
  refresh()

  #Mostrar la tabla
  display(df)

## Ingresar valores de venta

In [24]:
#Función para editar los valores de producto vendido, en el archivo local
def Sell():
  global df

  while True:
    #Solicitud de la cantidad de producto comprado
    cant = input("Ingrese la cantidad vendida: ")
    try:
      cant = int(cant)

      break

    except:
      print("Cantidad no reconocida.")
      
      verif = input("¿Desea volver a intentarlo? (Y/N): ")
      verif = verif.upper()

      if verif == "N":
        break

    #Solicitud de código del producto comprado
    code = input("Ingrese el código del producto: ")
    code = code.upper()

    #Actualización de la cantidad del producto comprado
    try:
      df.loc[df['Código'] == code, 'Venta'] += cant

      break

    except:
      print("Código no encontrado.")

      verif = input("¿Desea volver a intentarlo? (Y/N):")
      verif = verif.upper

      if verif == "N":
        break

  #Refrescar la tabla
  refresh()

  #Mostrar la tabla
  display(df)

## Ingresar valores de producto dañado

In [25]:
#Función para editar los valores de producto dañado, en el archivo local
def Damage():
  global df

  while True:
    #Solicitud de la cantidad de producto comprado
    cant = input("Ingrese la cantidad dañada: ")
    try:
      cant = int(cant)

      break

    except:
      print("Cantidad no reconocida.")
      
      verif = input("¿Desea volver a intentarlo? (Y/N): ")
      verif = verif.upper()

      if verif == "N":
        break

    #Solicitud de código del producto comprado
    code = input("Ingrese el código del producto: ")
    code = code.upper()

    #Actualización de la cantidad del producto comprado
    try:
      df.loc[df['Código'] == code, 'Dañado'] += cant

      break

    except:
      print("Código no encontrado.")

      verif = input("¿Desea volver a intentarlo? (Y/N):")
      verif = verif.upper

      if verif == "N":
        break

  #Refrescar la tabla
  refresh()

  #Mostrar la tabla
  display(df)

# Mostrar la tabla y actualizar

In [34]:
def Inventory():
    df, file_path = gen_file()

    display(df, max_rows=None, max_colums=None)
    while True:
        print("----------------------------------------")
        print("Opciones de edición del inventario:")
        print("1. Añadir nuevo producto")
        print("2. Eliminar producto")
        print("3. Registrar compra")
        print("4. Registrar venta")
        print("5. Registrar daño")
        print("6. Guardar cambios")
        print("7. Salir")
        choice = input("Seleccione una opción (1-7): ")
        if choice == '1':
            new_product()
        elif choice == '2':
            delete()
        elif choice == '3':
            Buy()
        elif choice == '4':
            Sell()
        elif choice == '5':
            Damage()
        elif choice == '6':
            Save_Changes()
        elif choice == "7":
            break
        else:
            print("Opción no reconocida.")

In [35]:
Inventory()

Archivo encontrado.


,Código,Nombre,Compra,Venta,Dañado,Stock
0,BQY41418,Crema hidratante con ácido hialurónico de Bioaqua,0,0,0,0
1,BQY01130,contorno para ojos con ácido hialurónico de Bi...,0,0,0,0
2,BQY58796,Jabón con ácido hialurónico de Bioaqua,0,0,0,0
3,BQY00057,Limpiador facial para el acné,0,0,0,0
4,BQY51278,Crema reparadora y eliminador de acné,0,0,0,0
5,BQY57621,Suero Esencia eliminador de acné,0,0,0,0
6,BQY13019,Limpiador facial hidratante de aloe vera,0,0,0,0
7,BQY04536,Tónico hidratante con aloe vera,0,0,0,0
8,BQY04581,Crema hidratante con aloe vera,0,0,0,0
9,BQY04598,Gel hidratante y suavizante de aloe vera,0,0,0,0


----------------------------------------
Opciones de edición del inventario:
1. Añadir nuevo producto
2. Eliminar producto
3. Registrar compra
4. Registrar venta
5. Registrar daño
6. Guardar cambios
7. Salir
